

### Generación de Modelos Predictivos con LOGISTIC REGRESSION

Este tipo de modelos requiere una revisión previa de colienalidad.  Tambien permiten obtener información del peso de las variables en el modelo.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

# Cargar tu DataFrame, reemplaza 'tu_dataframe.csv' con el nombre de tu archivo CSV
dfbase = pd.read_pickle('checkpoints/DATOS_LIMPIOS.pkl')


In [3]:
df = dfbase.copy()
# Dividir el DataFrame en variables independientes (X) y la variable dependiente (y)
X = df.drop('LABEL', axis=1)
y = df['LABEL']

In [ ]:
# calculamos correlación de variables independientes
corr_matrix = X.corr()

In [ ]:
# Inicializa una lista para almacenar las parejas de variables y sus valores de correlación
correlation_pairs = []

# Obtén las dimensiones de la matriz de correlación
num_rows, num_cols = corr_matrix.shape

# Itera a través de las columnas de la matriz de correlación
for i in range(num_cols):
    for j in range(i + 1, num_cols):  # Evita duplicados y la diagonal principal
        variable1 = corr_matrix.columns[i]
        variable2 = corr_matrix.columns[j]
        correlation_value = corr_matrix.iloc[i, j]
        correlation_pairs.append((variable1, variable2, correlation_value))

# Ordena la lista de parejas de variables por valor de correlación de mayor a menor
correlation_pairs.sort(key=lambda x: abs(x[2]), reverse=True)



In [10]:
dfcorr = pd.DataFrame(correlation_pairs,columns=['var1','var2','corr'])

In [19]:
temp = dfcorr.sort_values(by='corr', ascending=False)

# Filtra las filas donde 'var1' es diferente de 'var2'
dfcorr = temp[temp['var1'] != temp['var2']]



In [32]:
intervalos = [-1, 0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

# Utiliza pd.cut para dividir 'corr' en intervalos
dfcorr['intervalo'] = pd.cut(dfcorr['corr'].abs(), bins=intervalos)

# Agrupa y cuenta las filas en cada intervalo
tabla_resumen = dfcorr.groupby('intervalo')['corr'].agg([min, max, 'count']).reset_index()

# Renombra las columnas
tabla_resumen.columns = ['Intervalo', 'Mínimo', 'Máximo', 'Cantidad']

# Muestra la tabla resumen
print(tabla_resumen)

      Intervalo    Mínimo    Máximo  Cantidad
0   (-1.0, 0.0]       NaN       NaN         0
1    (0.0, 0.1] -0.099999  0.099998   1830411
2    (0.1, 0.2] -0.199931  0.199994     47540
3    (0.2, 0.3] -0.299742  0.299997     10871
4    (0.3, 0.4] -0.398727  0.399766      3977
5    (0.4, 0.5] -0.499354  0.499942      2562
6    (0.5, 0.6] -0.598363  0.599702      2135
7    (0.6, 0.7] -0.696373  0.699747       996
8    (0.7, 0.8] -0.792244  0.799353      1437
9    (0.8, 0.9] -0.857050  0.897663      1466
10   (0.9, 1.0] -1.000000  1.000000      4141


/var/folders/tx/1tx9w9fj6v78n5f12zkwg_vc0000gn/T/ipykernel_47254/4287402673.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla_resumen = dfcorr.groupby('intervalo')['corr'].agg([min, max, 'count']).reset_index()
/var/folders/tx/1tx9w9fj6v78n5f12zkwg_vc0000gn/T/ipykernel_47254/4287402673.py:7: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  tabla_resumen = dfcorr.groupby('intervalo')['corr'].agg([min, max, 'count']).reset_index()
/var/folders/tx/1tx9w9fj6v78n5f12zkwg_vc0000gn/T/ipykernel_47254/4287402673.py:7: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In

In [41]:
# Filtra las filas donde 'corr' está entre 0 y 0.1
resultado = dfcorr[(dfcorr['corr'] >= 0) & (dfcorr['corr'] <= 0.0001)]

# Crea un conjunto de variables únicas en 'var1' y 'var2' en las filas filtradas
conjunto_de_variables = set(resultado['var1']).union(set(resultado['var2']))

# Muestra el conjunto de variables resultante
print(len(conjunto_de_variables))

348


In [28]:


# Establece un umbral para la correlación
umbral_correlacion = 0.5

# Encuentra las columnas que tienen correlaciones por encima del umbral
columnas_eliminar = set()
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > umbral_correlacion:
            colname = corr_matrix.columns[i]
            columnas_eliminar.add(colname)

# Elimina las columnas que pasan el umbral
Xnew = X.drop(columns=columnas_eliminar)
print(X.shape)
print(Xnew.shape)

(7145, 1953)
(7145, 1022)


In [29]:

# Separar los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(Xnew, y, test_size=0.2, random_state=42)

# Aplicar StandardScaler para estandarizar las características (opcional pero recomendado)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [30]:

# Entrenar el modelo de regresión logística
model = LogisticRegression(max_iter=1000)
cv_scores = cross_val_score(model, X, y, cv=5)  # Utiliza 5 divisiones para validación cruzada

# Imprimir los puntajes de validación cruzada
print("Puntajes de validación cruzada:", cv_scores)
print("Precisión media: {:.2f}".format(cv_scores.mean()))

/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    h

Puntajes de validación cruzada: [0.74107768 0.70538838 0.7179846  0.75437369 0.71588523]
Precisión media: 0.73


/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
